<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Reflection" data-toc-modified-id="Reflection-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Reflection</a></span><ul class="toc-item"><li><span><a href="#Why-Reflection?" data-toc-modified-id="Why-Reflection?-12.1"><span class="toc-item-num">12.1&nbsp;&nbsp;</span>Why Reflection?</a></span></li><li><span><a href="#reflect.Type-and-reflect.Value" data-toc-modified-id="reflect.Type-and-reflect.Value-12.2"><span class="toc-item-num">12.2&nbsp;&nbsp;</span>reflect.Type and reflect.Value</a></span></li><li><span><a href="#Display,-a-Recursive-Value-Printer" data-toc-modified-id="Display,-a-Recursive-Value-Printer-12.3"><span class="toc-item-num">12.3&nbsp;&nbsp;</span>Display, a Recursive Value Printer</a></span></li><li><span><a href="#Example:-Encoding-S-Expressions" data-toc-modified-id="Example:-Encoding-S-Expressions-12.4"><span class="toc-item-num">12.4&nbsp;&nbsp;</span>Example: Encoding S-Expressions</a></span></li><li><span><a href="#Setting-Variables-with-reflect.Value" data-toc-modified-id="Setting-Variables-with-reflect.Value-12.5"><span class="toc-item-num">12.5&nbsp;&nbsp;</span>Setting Variables with reflect.Value</a></span></li><li><span><a href="#Example:-Decoding-S-Expressions" data-toc-modified-id="Example:-Decoding-S-Expressions-12.6"><span class="toc-item-num">12.6&nbsp;&nbsp;</span>Example: Decoding S-Expressions</a></span></li><li><span><a href="#Accessing-Struct-Field-Tags" data-toc-modified-id="Accessing-Struct-Field-Tags-12.7"><span class="toc-item-num">12.7&nbsp;&nbsp;</span>Accessing Struct Field Tags</a></span></li><li><span><a href="#Displaying-the-Methods-of-a-Type" data-toc-modified-id="Displaying-the-Methods-of-a-Type-12.8"><span class="toc-item-num">12.8&nbsp;&nbsp;</span>Displaying the Methods of a Type</a></span></li></ul></li></ul></div>

# Reflection

## Why Reflection?

In [1]:
func Sprint(x interface{}) string{
    type stringer interface {
        String() string
    }
    switch x := x.(type) {
        case stringer:
          return x.String()
        case string:
          return x
        case int:
          return strconv.Itoa(x) // ... similar cases for int16, uint32, and so on...
        case bool:
          if x {
            return "true"
          }
          return "false"
        defaut:
          // array, chan, func, map, pointer, slice, struct
          return "???"
    }
}

ERROR: repl.go:11:18: undefined "strconv" in strconv.Itoa <*ast.SelectorExpr>

## reflect.Type and reflect.Value

In [6]:
import "fmt"
t := reflect.TypeOf(3) // a reflect.Type
fmt.Println(t.String())// "int"
fmt.Println(t)         // "int"

int
int


4 <nil>

In [7]:
import ("io";"os")
var w io.Writer = os.Stdout 
fmt.Println(reflect.TypeOf(w))

*os.File


9 <nil>

In [8]:
fmt.Printf("%T\n",3)   // "int"

int


4 <nil>

In [10]:
v := reflect.ValueOf(3)
fmt.Println(v)          // "3"
fmt.Printf("%v\n",v)    // "3"
fmt.Println(v.String()) // <int Value>
fmt.Println(v)

3
3
<int Value>
3


2 <nil>

In [7]:
package format

import (
        "reflect"
        "strconv"
)

// Any formats any value as a string.
func Any(value interface{}) string {
        return formatAtom(reflect.ValueOf(value))
}

// formatAtom formats a value without inspecting its internal structure.
func formatAtom(v reflect.Value) string {
        switch v.Kind() {
        case reflect.Invalid:
                return "invalid"
        case reflect.Int, reflect.Int8, reflect.Int16,
                reflect.Int32, reflect.Int64:
                return strconv.FormatInt(v.Int(), 10)
        case reflect.Uint, reflect.Uint8, reflect.Uint16,
                reflect.Uint32, reflect.Uint64, reflect.Uintptr:
                return strconv.FormatUint(v.Uint(), 10)
            // ...floating-point and complex cases omitted for brevity...
        case reflect.Bool:
                return strconv.FormatBool(v.Bool())
        case reflect.String:
                return strconv.Quote(v.String())
        case reflect.Chan, reflect.Func, reflect.Ptr, reflect.Slice, reflect.Map:
                return v.Type().String() + " 0x" +
                        strconv.FormatUint(uint64(v.Pointer()), 16)
        default: // reflect.Array, reflect.Struct, reflect.Interface
                return v.Type().String() + " value"
        }
}

In [13]:
import ("time";"gopl.io/ch12/format";"reflect";"fmt")
var x int64 = 1
var d time.Duration = 1 * time.Nanosecond
fmt.Println(format.Any(x))
fmt.Println(format.Any(d))
fmt.Println(format.Any([]int64{x}))
fmt.Println(format.Any([]time.Duration{d}))

1
1
[]int64 0xc0000c5780
[]time.Duration 0xc0000c57a0


29 <nil>

## Display, a Recursive Value Printer

In [8]:
import "gopl.io/ch12/format"
func display(path string, v reflect.Value) {
        switch v.Kind() {
        case reflect.Invalid:
                fmt.Printf("%s = invalid\n", path)
        case reflect.Slice, reflect.Array:
                for i := 0; i < v.Len(); i++ {
                        display(fmt.Sprintf("%s[%d]", path, i), v.Index(i))
                }
        case reflect.Struct:
                for i := 0; i < v.NumField(); i++ {
                        fieldPath := fmt.Sprintf("%s.%s", path, v.Type().Field(i).Name)
                        display(fieldPath, v.Field(i))
                }
        case reflect.Map:
                for _, key := range v.MapKeys() {
                        display(fmt.Sprintf("%s[%s]", path,
                                formatAtom(key)), v.MapIndex(key))
                }
        case reflect.Ptr:
                if v.IsNil() {
                        fmt.Printf("%s = nil\n", path)
                } else {
                        display(fmt.Sprintf("(*%s)", path), v.Elem())
                }
        case reflect.Interface:
                if v.IsNil() {
                        fmt.Printf("%s = nil\n", path)
                } else {
                        fmt.Printf("%s.type = %s\n", path, v.Elem().Type())
                        display(path+".value", v.Elem())
                }
        default: // basic types, channels, funcs
                fmt.Printf("%s = %s\n", path, formatAtom(v))
        }
}                

In [9]:
import ("fmt";"reflect")
func Display(name string, x interface{}){
    fmt.Print("Display %s (%T):\n",name, x)
    display(name, reflect.ValueOf(x))
}

In [2]:
import ("gopl.io/ch12/display"; "gopl.io/ch7/eval")
e, _ := eval.Parse("sqrt(A/pi)")
display.Display("e",e)
display.Display("slice", []*int{new(int), nil})

Display e (eval.call):
e.fn = "sqrt"
e.args[0].type = eval.binary
e.args[0].value.op = 47
e.args[0].value.x.type = eval.Var
e.args[0].value.x.value = "A"
e.args[0].value.y.type = eval.Var
e.args[0].value.y.value = "pi"
Display slice ([]*int):
(*slice[0]) = 0
slice[1] = nil


In [5]:
import "io"
var w io.Writer
display.Display("w", w)
var w io.Writer
display.Display("&w", &w)

Display w (<nil>):
w = invalid
Display &w (*io.Writer):
(*&w) = nil


In [13]:
   type Movie struct {
                Title, Subtitle string
                Year            int
                Color           bool
                Actor           map[string]string
                Oscars          []string
                Sequel          *string
        }
        //!-movie
        //!+strangelove
        strangelove := Movie{
                Title:    "Dr. Strangelove",
                Subtitle: "How I Learned to Stop Worrying and Love the Bomb",
                Year:     1964,
                Color:    false,
                Actor: map[string]string{
                        "Dr. Strangelove":            "Peter Sellers",
                        "Grp. Capt. Lionel Mandrake": "Peter Sellers",
                        "Pres. Merkin Muffley":       "Peter Sellers",
                        "Gen. Buck Turgidson":        "George C. Scott",
                        "Brig. Gen. Jack D. Ripper":  "Sterling Hayden",
                        `Maj. T.J. "King" Kong`:      "Slim Pickens",
                },
             Oscars: []string{
                        "Best Actor (Nomin.)",
                        "Best Adapted Screenplay (Nomin.)",
                        "Best Director (Nomin.)",
                        "Best Picture (Nomin.)",
                },
        }
        //!-strangelove
        display.Display("strangelove", strangelove)

Display strangelove (struct { Title string; Subtitle string; Year int; Color bool; Actor map[string]string; Oscars []string; Sequel *string }):
strangelove.Title = "Dr. Strangelove"
strangelove.Subtitle = "How I Learned to Stop Worrying and Love the Bomb"
strangelove.Year = 1964
strangelove.Color = false
strangelove.Actor["Dr. Strangelove"] = "Peter Sellers"
strangelove.Actor["Grp. Capt. Lionel Mandrake"] = "Peter Sellers"
strangelove.Actor["Pres. Merkin Muffley"] = "Peter Sellers"
strangelove.Actor["Gen. Buck Turgidson"] = "George C. Scott"
strangelove.Actor["Brig. Gen. Jack D. Ripper"] = "Sterling Hayden"
strangelove.Actor["Maj. T.J. \"King\" Kong"] = "Slim Pickens"
strangelove.Oscars[0] = "Best Actor (Nomin.)"
strangelove.Oscars[1] = "Best Adapted Screenplay (Nomin.)"
strangelove.Oscars[2] = "Best Director (Nomin.)"
strangelove.Oscars[3] = "Best Picture (Nomin.)"
strangelove.Sequel = nil


In [8]:
import "os"   
display.Display("os.Stderr", os.Stderr)

Display os.Stderr (*os.File):
(*(*os.Stderr).file).pfd.fdmu.state = 0
(*(*os.Stderr).file).pfd.fdmu.rsema = 0
(*(*os.Stderr).file).pfd.fdmu.wsema = 0
(*(*os.Stderr).file).pfd.Sysfd = 22
(*(*os.Stderr).file).pfd.pd.runtimeCtx = 140160623848312
(*(*os.Stderr).file).pfd.iovecs = nil
(*(*os.Stderr).file).pfd.csema = 0
(*(*os.Stderr).file).pfd.isBlocking = 0
(*(*os.Stderr).file).pfd.IsStream = true
(*(*os.Stderr).file).pfd.ZeroReadIsEOF = true
(*(*os.Stderr).file).pfd.isFile = true
(*(*os.Stderr).file).name = "|1"
(*(*os.Stderr).file).dirinfo = nil
(*(*os.Stderr).file).nonblock = true
(*(*os.Stderr).file).stdoutOrErr = false
(*(*os.Stderr).file).appendMode = false


## Example: Encoding S-Expressions

In [14]:
import (
        "bytes"
        "fmt"
        "reflect"
)
func Marshal(v interface{}) ([]byte, error) {
        var buf bytes.Buffer
        if err := encode(&buf, reflect.ValueOf(v)); err != nil {
                return nil, err
        }
        return buf.Bytes(), nil
}
func encode(buf *bytes.Buffer, v reflect.Value) error {
        switch v.Kind() {
        case reflect.Invalid:
                buf.WriteString("nil")

        case reflect.Int, reflect.Int8, reflect.Int16,
                reflect.Int32, reflect.Int64:
                fmt.Fprintf(buf, "%d", v.Int())

        case reflect.Uint, reflect.Uint8, reflect.Uint16,
                reflect.Uint32, reflect.Uint64, reflect.Uintptr:
                fmt.Fprintf(buf, "%d", v.Uint())

        case reflect.String:
                fmt.Fprintf(buf, "%q", v.String())

        case reflect.Ptr:
                return encode(buf, v.Elem())

        case reflect.Array, reflect.Slice: // (value ...)
                buf.WriteByte('(')
                for i := 0; i < v.Len(); i++ {
                        if i > 0 {
                             buf.WriteByte(' ')
                        }
                        if err := encode(buf, v.Index(i)); err != nil {
                                return err
                        }
                }
                buf.WriteByte(')')

        case reflect.Struct: // ((name value) ...)
                buf.WriteByte('(')
                for i := 0; i < v.NumField(); i++ {
                        if i > 0 {
                                buf.WriteByte(' ')
                        }
                        fmt.Fprintf(buf, "(%s ", v.Type().Field(i).Name)
                        if err := encode(buf, v.Field(i)); err != nil {
                                return err
                        }
                        buf.WriteByte(')')
                }
                buf.WriteByte(')')

        case reflect.Map: // ((key value) ...)
            buf.WriteByte('(')
                for i, key := range v.MapKeys() {
                        if i > 0 {
                                buf.WriteByte(' ')
                        }
                        buf.WriteByte('(')
                        if err := encode(buf, key); err != nil {
                                return err
                        }
                        buf.WriteByte(' ')
                        if err := encode(buf, v.MapIndex(key)); err != nil {
                                return err
                        }
                        buf.WriteByte(')')
                }
                buf.WriteByte(')')

        default: // float, complex, bool, chan, func, interface
                return fmt.Errorf("unsupported type: %s", v.Type())
        }
        return nil
}

In [27]:
import "gopl.io/ch12/sexpr"
data, err := sexpr.Marshal(strangelove)
fmt.Printf("%s", data)

ERROR: repl.go:2:28: undefined identifier: strangelove

## Setting Variables with reflect.Value

In [41]:
x := 2                   // value   type   variable
a := reflect.ValueOf(2)  // 2       int    no
b := reflect.ValueOf(x)  // 2       int    no
c := reflect.ValueOf(&x) // &x      *int   no
d := c.Elem()            // 2       int    yes (x)

2

In [42]:
fmt.Println(a.CanAddr()) 
fmt.Println(b.CanAddr())
fmt.Println(c.CanAddr())
fmt.Println(d.CanAddr())

false
false
false
true


5 <nil>

In [43]:
x := 2
d := reflect.ValueOf(&x).Elem()
px := d.Addr().Interface().(*int)
*px = 3
fmt.Println(x)

3


2 <nil>

In [44]:
d.Set(reflect.ValueOf(4))
fmt.Println(x)  

4


2 <nil>

In [45]:
d.Set(reflect.ValueOf(int64(5)))

ERROR: reflect.Set: value of type int64 is not assignable to type int

In [46]:
x := 2
b := reflect.ValueOf(x)
b.Set(reflect.Value(3))  

ERROR: cannot convert untyped constant {int 3} to <reflect.Value>

In [48]:
d := reflect.ValueOf(&x).Elem()
d.SetInt(3)
fmt.Println(x)

3


2 <nil>

In [25]:
x := 1
rx := reflect.ValueOf(&x).Elem()
rx.SetInt(2)
rx.Set(reflect.ValueOf(3))

In [26]:
rx.SetString("hello")
rx.Set(reflect.ValueOf("hello"))

ERROR: reflect: call of reflect.Value.SetString on int Value

In [55]:
var y interface{}
ry := reflect.ValueOf(&y).Elem()
//ry.SetInt(2)
//ry.Set(reflect.ValueOf(3))
//ry.SetString("hello")
ry.Set(reflect.ValueOf("hello"))

In [63]:
stdout := reflect.ValueOf(os.Stdout).Elem()
fmt.Println(stdout.Type())
fmt.Printf("%q", stdout)
fd := stdout.FieldByName("fd")
//fmt.Println(fd.Int())
fmt.Println(fd.CanAddr(),fd.CanSet())

os.File
{%!q(*os.file=&{{{0 0 0} 24 {140160623845992} <nil> 0 0 true true true} |1 <nil> true false false})}false false


12 <nil>

In [64]:
import "gopl.io/ch12/display"
display.Display("os.stdout", stdout)

Display os.stdout (reflect.Value):
(*os.stdout.typ).size = 8
(*os.stdout.typ).ptrdata = 8
(*os.stdout.typ).hash = 1205328019
(*os.stdout.typ).tflag = 15
(*os.stdout.typ).align = 8
(*os.stdout.typ).fieldAlign = 8
(*os.stdout.typ).kind = 57
(*os.stdout.typ).equal = func(unsafe.Pointer, unsafe.Pointer) bool 0x80f1a0
(*(*os.stdout.typ).gcdata) = 1
(*os.stdout.typ).str = 26948
(*os.stdout.typ).ptrToThis = 2002784
os.stdout.ptr = unsafe.Pointer value
os.stdout.flag = 409


In [66]:
display.Display("os.stdout",os.Stdout)

Display os.stdout (*os.File):
(*(*os.stdout).file).pfd.fdmu.state = 0
(*(*os.stdout).file).pfd.fdmu.rsema = 0
(*(*os.stdout).file).pfd.fdmu.wsema = 0
(*(*os.stdout).file).pfd.Sysfd = 20
(*(*os.stdout).file).pfd.pd.runtimeCtx = 140160623848776
(*(*os.stdout).file).pfd.iovecs = nil
(*(*os.stdout).file).pfd.csema = 0
(*(*os.stdout).file).pfd.isBlocking = 0
(*(*os.stdout).file).pfd.IsStream = true
(*(*os.stdout).file).pfd.ZeroReadIsEOF = true
(*(*os.stdout).file).pfd.isFile = true
(*(*os.stdout).file).name = "|1"
(*(*os.stdout).file).dirinfo = nil
(*(*os.stdout).file).nonblock = true
(*(*os.stdout).file).stdoutOrErr = false
(*(*os.stdout).file).appendMode = false


## Example: Decoding S-Expressions

In [13]:
import (
        "bytes"
        "fmt"
        "reflect"
        "strconv"
        "text/scanner"
)
//!+lexer
type lexer struct {
        scan  scanner.Scanner
        token rune // the current token
}

func (lex *lexer) next()        { lex.token = lex.scan.Scan() }
func (lex *lexer) text() string { return lex.scan.TokenText() }

func (lex *lexer) consume(want rune) {
        if lex.token != want { // NOTE: Not an example of good error handling.
                panic(fmt.Sprintf("got %q, want %q", lex.text(), want))
        }
        lex.next()
}

In [24]:
func read(lex *lexer, v reflect.Value) {
        switch lex.token {
        case scanner.Ident:
                // The only valid identifiers are
                // "nil" and struct field names.
                if lex.text() == "nil" {
                        v.Set(reflect.Zero(v.Type()))
                        lex.next()
                        return
                }
        case scanner.String:
                s, _ := strconv.Unquote(lex.text()) // NOTE: ignoring errors
                v.SetString(s)
                lex.next()
                return
        case scanner.Int:
                i, _ := strconv.Atoi(lex.text()) // NOTE: ignoring errors
                v.SetInt(int64(i))
                lex.next()
                return
        case '(':
                lex.next()
                readList(lex, v)
                lex.next() // consume ')'
                return
        }
        panic(fmt.Sprintf("unexpected token %q", lex.text()))
}
func readList(lex *lexer, v reflect.Value) {
        switch v.Kind() {
        case reflect.Array: // (item ...)
                for i := 0; !endList(lex); i++ {
                        read(lex, v.Index(i))
                }

        case reflect.Slice: // (item ...)
                for !endList(lex) {
                        item := reflect.New(v.Type().Elem()).Elem()
                        read(lex, item)
                        v.Set(reflect.Append(v, item))
                }

        case reflect.Struct: // ((name value) ...)
                for !endList(lex) {
                        lex.consume('(')
                        if lex.token != scanner.Ident {
                                panic(fmt.Sprintf("got token %q, want field name", lex.text()))
                        }
                        name := lex.text()
                        lex.next()
                        read(lex, v.FieldByName(name))
                        lex.consume(')')
                }

        case reflect.Map: // ((key value) ...)
                v.Set(reflect.MakeMap(v.Type()))
                for !endList(lex) {
                        lex.consume('(')
                        key := reflect.New(v.Type().Key()).Elem()
                        read(lex, key)
                        value := reflect.New(v.Type().Elem()).Elem()
                        read(lex, value)
                        v.SetMapIndex(key, value)
                        lex.consume(')')
                }

        default:
                panic(fmt.Sprintf("cannot decode list into %v", v.Type()))
        }
}

ERROR: repl.go:33:25: undefined identifier: read

In [17]:
func endList(lex *lexer) bool {
        switch lex.token {
        case scanner.EOF:
                panic("end of file")
        case ')':
                return true
        }
        return false
}

## Accessing Struct Field Tags

In [11]:
import "net/http"
import "gopl.io/ch12/params"

// search implements the /search URL endpoint.
func search(resp http.ResponseWriter, req *http.Request) {
        var data struct {
                Labels     []string `http:"l"`
                MaxResults int      `http:"max"`
                Exact      bool     `http:"x"`
        }
        data.MaxResults = 10 // set default
        if err := params.Unpack(req, &data); err != nil {
                http.Error(resp, err.Error(), http.StatusBadRequest) // 400
                return
        }

        // ...rest of handler...
        fmt.Fprintf(resp, "Search: %+v\n", data)
}

In [10]:
import "log"
func main() {
        http.HandleFunc("/search", search)
        log.Fatal(http.ListenAndServe(":12345", nil))
}

/*
//!+output
$ go build gopl.io/ch12/search
$ ./search &
$ ./fetch 'http://localhost:12345/search'
Search: {Labels:[] MaxResults:10 Exact:false}
$ ./fetch 'http://localhost:12345/search?l=golang&l=programming'
Search: {Labels:[golang programming] MaxResults:10 Exact:false}
$ ./fetch 'http://localhost:12345/search?l=golang&l=programming&max=100'
Search: {Labels:[golang programming] MaxResults:100 Exact:false}
$ ./fetch 'http://localhost:12345/search?x=true&l=golang&l=programming'
Search: {Labels:[golang programming] MaxResults:10 Exact:true}
$ ./fetch 'http://localhost:12345/search?q=hello&x=123'
x: strconv.ParseBool: parsing "123": invalid syntax
$ ./fetch 'http://localhost:12345/search?q=hello&max=lots'
max: strconv.ParseInt: parsing "lots": invalid syntax
//!-output
*/

In [ ]:
main()

* [http://localhost:12345/search](http://localhost:12345/search)
* [http://localhost:12345/search?l=golang&l=programming](http://localhost:12345/search?l=golang&l=programming)
* [http://localhost:12345/search?l=golang&l=programming&max=100](http://localhost:12345/search?l=golang&l=programming&max=100)
* [http://localhost:12345/search?x=true&l=golang&l=programming](http://localhost:12345/search?x=true&l=golang&l=programming)
* [http://localhost:12345/search?q=hello&x=123](http://localhost:12345/search?q=hello&x=123)
* [http://localhost:12345/search?q=hello&max=lots](http://localhost:12345/search?q=hello&max=lots)

In [9]:
//!+populate
import "strconv"
import "fmt"
func populate(v reflect.Value, value string) error {
        switch v.Kind() {
        case reflect.String:
                v.SetString(value)

        case reflect.Int:
                i, err := strconv.ParseInt(value, 10, 64)
                if err != nil {
                        return err
                }
                v.SetInt(i)

        case reflect.Bool:
                b, err := strconv.ParseBool(value)
                if err != nil {
                        return err
                }
                v.SetBool(b)

        default:
                return fmt.Errorf("unsupported kind %s", v.Type())
        }
        return nil
}

In [10]:
import "net/http"
import "reflect"
import "strings"
// Unpack populates the fields of the struct pointed to by ptr
// from the HTTP request parameters in req.
func Unpack(req *http.Request, ptr interface{}) error {
        if err := req.ParseForm(); err != nil {
                return err
        }

        // Build map of fields keyed by effective name.
        fields := make(map[string]reflect.Value)
        v := reflect.ValueOf(ptr).Elem() // the struct variable
        for i := 0; i < v.NumField(); i++ {
                fieldInfo := v.Type().Field(i) // a reflect.StructField
                tag := fieldInfo.Tag           // a reflect.StructTag
                name := tag.Get("http")
                if name == "" {
                        name = strings.ToLower(fieldInfo.Name)
                }
                fields[name] = v.Field(i)
        }

        // Update struct field for each parameter in the request.
        for name, values := range req.Form {
                f := fields[name]
                if !f.IsValid() {
                        continue // ignore unrecognized HTTP parameters
                }
                for _, value := range values {
                        if f.Kind() == reflect.Slice {
                                elem := reflect.New(f.Type().Elem()).Elem()
                                if err := populate(elem, value); err != nil {
                                        return fmt.Errorf("%s: %v", name, err)
                                }
                                f.Set(reflect.Append(f, elem))
                         } else {
                                if err := populate(f, value); err != nil {
                                        return fmt.Errorf("%s: %v", name, err)
                                }
                        }
                }
        }
        return nil
}                    

## Displaying the Methods of a Type

In [30]:
import "strings"
import "reflect"
import "fmt"
func Print(x interface{}) {
        v := reflect.ValueOf(x)
        t := v.Type()
        fmt.Printf("type %s\n", t)

        for i := 0; i < v.NumMethod(); i++ {
                methType := v.Method(i).Type()
                fmt.Printf("func (%s) %s%s\n", t, t.Method(i).Name,
                        strings.TrimPrefix(methType.String(), "func"))
        }
}

In [31]:
import "time"
Print(time.Hour)

type time.Duration
func (time.Duration) Hours() float64
func (time.Duration) Microseconds() int64
func (time.Duration) Milliseconds() int64
func (time.Duration) Minutes() float64
func (time.Duration) Nanoseconds() int64
func (time.Duration) Round(time.Duration) time.Duration
func (time.Duration) Seconds() float64
func (time.Duration) String() string
func (time.Duration) Truncate(time.Duration) time.Duration


In [32]:
import "strings"
Print(new(strings.Replacer))

type *strings.Replacer
func (*strings.Replacer) Replace(string) string
func (*strings.Replacer) WriteString(io.Writer, string) (int, error)
